In [1]:
import pandas as pd
import requests
from pandas.io.json import json_normalize
from sqlalchemy import create_engine

In [2]:
#loading GDP data
gdp_data_df = pd.read_csv("gdp.csv")
gdp_data_df.head()

,Country Code,Country Name,Year,Value
0,ARB,Arab World,1968,25760683041
1,ARB,Arab World,1969,28434203615
2,ARB,Arab World,1970,31385499664
3,ARB,Arab World,1971,36426909888
4,ARB,Arab World,1972,43316056615


In [3]:
#loading world corrruption index csv
CPI_data_allyears_df = pd.read_csv("cpi_complete.csv", encoding='ISO-8859-1', engine='python')
CPI_data_allyears_df.head()

,country_code,Country,Region,CPI score 2016,CPI score 2015,CPI score 2014,CPI Score 2013,CPI Score 2012
0,DNK,Denmark,WE/EU,90.0,91.0,92.0,91.0,90.0
1,NZL,New Zealand,AP,90.0,91.0,91.0,91.0,90.0
2,FIN,Finland,WE/EU,89.0,90.0,89.0,89.0,90.0
3,SGP,Singapore,AP,84.0,85.0,84.0,86.0,87.0
4,SWE,Sweden,WE/EU,88.0,89.0,87.0,89.0,88.0


In [4]:
#.copy method for CPI dataframe 1 
CPI_data_copy_ay = CPI_data_allyears_df[['country_code','Country','Region','CPI score 2016','CPI score 2015','CPI score 2014','CPI Score 2013','CPI Score 2012']].copy()
CPI_data_copy_ay.head()

,country_code,Country,Region,CPI score 2016,CPI score 2015,CPI score 2014,CPI Score 2013,CPI Score 2012
0,DNK,Denmark,WE/EU,90.0,91.0,92.0,91.0,90.0
1,NZL,New Zealand,AP,90.0,91.0,91.0,91.0,90.0
2,FIN,Finland,WE/EU,89.0,90.0,89.0,89.0,90.0
3,SGP,Singapore,AP,84.0,85.0,84.0,86.0,87.0
4,SWE,Sweden,WE/EU,88.0,89.0,87.0,89.0,88.0


In [5]:
#transforming GDP data for postgress inject
gdp_data_clean = gdp_data_df.rename(columns={"Country Code": "country_code",
                                             "Country Name": "country_name",
                                             "Year": "reporting_year",
                                             "Value": "mon_value"})
gdp_data_clean.set_index("country_code", inplace=True)

gdp_data_clean.head()

,country_name,reporting_year,mon_value
country_code,,,
ARB,Arab World,1968,25760683041
ARB,Arab World,1969,28434203615
ARB,Arab World,1970,31385499664
ARB,Arab World,1971,36426909888
ARB,Arab World,1972,43316056615


In [6]:
#transforming CPI data ay for postgress inject

cpi_data_ay_clean = CPI_data_copy_ay.rename(columns={"Country Code": "country_code",
                                             "Country": "country_name",
                                             "Region": "region",
                                             "CPI score 2016": "cpi_2016",
                                             "CPI score 2015": "cpi_2015",
                                             "CPI score 2014": "cpi_2014",
                                             "CPI Score 2013": "cpi_2013",
                                             "CPI Score 2012": "cpi_2012"})
cpi_data_ay_clean.set_index("country_code", inplace=True)

cpi_data_ay_clean.head()

,country_name,region,cpi_2016,cpi_2015,cpi_2014,cpi_2013,cpi_2012
country_code,,,,,,,
DNK,Denmark,WE/EU,90.0,91.0,92.0,91.0,90.0
NZL,New Zealand,AP,90.0,91.0,91.0,91.0,90.0
FIN,Finland,WE/EU,89.0,90.0,89.0,89.0,90.0
SGP,Singapore,AP,84.0,85.0,84.0,86.0,87.0
SWE,Sweden,WE/EU,88.0,89.0,87.0,89.0,88.0


In [10]:
#establishing rds connections
rds_connection_string = "postgres:postgres@localhost:5432/world_corruption_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [14]:
#confirm tables
engine.table_names()

['gdp_data', 'world_cpi']

In [12]:
#loading GDP dataframe into created postgress database
gdp_data_clean.to_sql(name='gdp_data', con=engine, if_exists='append', index=True)

In [13]:
cpi_data_ay_clean.to_sql(name='world_cpi', con=engine, if_exists='append', index=True)

In [15]:
#Examples of executable queries
```sql
SELECT * FROM gdp_data
SELECT * FROM world_cpi

#query used to filter data by 2012 corruption index and 2012 gdp.
SELECT gdp_data.country_code,world_cpi.country_name, gdp_data.mon_value, world_cpi.cpi_2012,
gdp_data.reporting_year
FROM gdp_data
INNER JOIN world_cpi
ON gdp_data.country_code = world_cpi.country_code
WHERE reporting_year = '2012'
```

SyntaxError: invalid syntax (<ipython-input-15-a8005c7c4ccf>, line 2)